This script is an experiment in analyzing multi-hazard impacts on the power grid

The goal of this script is to identify a candidate set of wildfire + space weather events

Approach (implemented)
- Get wildfire point 
- Choose radius of some sort
- Get any power grid disturbance that occurred within that time and circle
- Inspect space weather data for preceding day to current time
- Store full multi-hazard data for the candidate event and study forensically

Possibility of a different approach (not implemented):
- Read in set of wildfire events from MODIS or Suomi
- Get time period, get spatial extent (GIS) -> visualize it
- Highest level: get space weather indices
    - more granular is to get magnetometers and GNSS data (GIMs, likely) in the area of the fire 
- Get HIFLD grid affected based on the wildfire
- (optional) get observed power grid disturbances that overlap in space and time
- Record the candidate event for forensic analysis (social media, interviews, grey literature exploration)


TODO
- the connection may very well come from space weather effects on communications during wildfire events (need to incorporate radiation data to represent space weather) 
- explore emdat database
- add sophistication to wildfire region processing
- add terrestrial weather to search
- explore 2024 data

#### Dependencies and Custom Functions

In [ ]:
import numpy as np
import pandas as pd
import os, sys

from datetime import datetime, time
import geopandas as gpd

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter

from shapely.geometry import Polygon, MultiPolygon

# import contextily as ctx


from supermag_api.supermag_api import *


import re


In [ ]:
def get_sm_one_minute(start,end):

    '''

    
    Dependencies
        supermag-api downloadable from https://supermag.jhuapl.edu/mag/?fidelity=low&start=2001-01-01T00%3A00%3A00.000Z&interval=1%3A00%3A00&tab=api
            --> one should move this to the working directory and rename to supermag_api
        datetime
            
    Notes
        it appears this will only work for a few-day request; for longer periods, need to download the data directly
            
    '''


#     import supermag_api
#     import datetime

    start_datetime = pd.to_datetime(start)
    end_datetime = pd.to_datetime(end)
    
    # get the number of days between start and end
    num_days = (end_datetime - start_datetime).days
    print('the number of days that will be requested is {}'.format(num_days))
    input('Press Enter if you are sure you want to ping the SuperMAG API for this...')
    
    # get correct format for 'start date'
    # date_obj_start = datetime.strptime(start, '%Y-%m-%d %H:%M:%S')
    # formatted_start = date_obj_start.strftime('%Y-%m-%dT%H:%M')
    date_obj_start = datetime.datetime.strptime(start, '%Y-%m-%d %H:%M:%S')
    formatted_start = date_obj_start.strftime('%Y-%m-%dT%H:%M')

    (status,df_smidxs) = SuperMAGGetIndices('rymc1012',formatted_start,86400*num_days,'SME, SMU, SML, SMR')

    datetimes_sm = [pd.to_datetime(t, unit='s') for t in df_smidxs['tval']]

    df_smidxs['datetimes'] = datetimes_sm
    df_smidxs = df_smidxs.set_index(['datetimes'])
    df_smidxs.index = pd.to_datetime(df_smidxs.index)
    
    return df_smidxs





#### Wildfire database

Some things I'm learning about wildfire data
- MODIS might be best available given coverage
- There is a NASA digital twin project https://ideas-digitaltwin.jpl.nasa.gov/airquality/, but it is focused on air quality and thus the atmospheric emissions associated with wildfire, whereas for this work I need the wildfires themselves (not their emissions which then mix with the atmosphere and its dynamics)
- ...


In [ ]:
# NASA FIRMS data https://firms.modaps.eosdis.nasa.gov


# National Interagency Fire Center https://www.nifc.gov/
file_path_nifc_wildfire = "/Users/ryanmc/Documents/Conferences/Jack_Eddy_Symposium_2022/dev/wildfire_data/WFIGS_Incident_Locations_6818922478522849878.geojson"
file_path_modis_wildfire = "/Users/ryanmc/Documents/Conferences/Jack_Eddy_Symposium_2022/dev/wildfire_data/DL_FIRE_M-C61_530609/fire_archive_M-C61_530609.shp"
# file_path_suomi_wildfire = "/Users/ryanmc/Documents/Conferences/Jack_Eddy_Symposium_2022/dev/wildfire_data/DL_FIRE_SV-C2_530613/fire_archive_SV-C2_530613.shp"

# gdf_nifc_wildfire = gpd.read_file(file_path_nifc_wildfire)
gdf_modis_wildfire = gpd.read_file(file_path_modis_wildfire)
# gdf_suomi_wildfire = gpd.read_file(file_path_suomi_wildfire)




In [ ]:
print('size of the wildfire data is: {:.2f} MB'.format(sys.getsizeof(gdf_modis_wildfire) / 10**6))

In [ ]:
# Explore the first few rows
print(gdf_modis_wildfire.head())

# Check the geometry type (e.g., Point, Polygon, LineString)
print(gdf_modis_wildfire.geom_type.unique())

# Get summary of the attributes
print(gdf_modis_wildfire.info())

# # Plot the shapefile (quick visualization)
# gdf.plot()


In [ ]:
if 'ACQ_DATE' in gdf_modis_wildfire.columns and 'ACQ_TIME' in gdf_modis_wildfire.columns:
    gdf_modis_wildfire['datetime'] = (
        gdf_modis_wildfire['ACQ_DATE'].astype(str) + ' ' + gdf_modis_wildfire['ACQ_TIME'].astype(str)
    )
    gdf_modis_wildfire['datetime'] = pd.to_datetime(gdf_modis_wildfire['datetime'])
else:
    raise ValueError("Expected 'ACQ_DATE' and 'ACQ_TIME' columns.")

### (optional) visualize the wildfire data

In [ ]:
import folium
from folium.plugins import TimestampedGeoJson



In [ ]:



# Convert GeoDataFrame to GeoJSON-like dictionary with timestamp for Folium
features = []
for _, row in gdf_modis_wildfire.iterrows():
    feature = {
        'type': 'Feature',
        'geometry': row['geometry'].__geo_interface__,
        'properties': {
            'time': row['datetime'].isoformat(),  # Use ISO format for timestamps
            'popup': f"Acquired: {row['datetime']}"
        }
    }
    features.append(feature)

# Create the Timestamped GeoJSON layer
timestamped_geojson = TimestampedGeoJson(
    {'type': 'FeatureCollection', 'features': features},
    period='PT1H',  # Period for animation: 1 hour
    add_last_point=False,
    auto_play=False,
    loop=True,
    max_speed=1,
    loop_button=True,
    date_options='YYYY-MM-DD HH:mm:ss',
    time_slider_drag_update=True
)

# Initialize a Folium map centered on the dataset's centroid
m = folium.Map(
    location=[gdf_modis_wildfire.geometry.y.mean(), gdf_modis_wildfire.geometry.x.mean()],
    zoom_start=6, tiles='CartoDB positron'
)

# Add the timestamped layer to the map
timestamped_geojson.add_to(m)

# Save the map as an HTML file and display it
m.save('interactive_map.html')

# If running in Jupyter, display the map directly
m


### Create a list of features grouped by unique timestamps


In [ ]:
features = []
for dt, group in gdf_modis_wildfire.groupby('datetime'):
    feature = {
        'type': 'FeatureCollection',
        'features': [
            {
                'type': 'Feature',
                'geometry': row['geometry'].__geo_interface__,
                'properties': {
                    'time': dt.isoformat(),  # Set timestamp in ISO format
                    'popup': f"Acquired: {dt}"
                }
            }
            for _, row in group.iterrows()
        ]
    }
    features.append(feature)


# # Create the TimestampedGeoJson with these features
# timestamped_geojson = TimestampedGeoJson(
#     {'type': 'FeatureCollection', 'features': features},
#     period='PT1H',              # Set period of time for each step (1 hour)
#     add_last_point=False,        # Ensure only current points appear
#     auto_play=False,              # Automatically start animation
#     loop=True,                   # Loop the animation
#     max_speed=1,                 # Normal animation speed
#     loop_button=True,            # Allow user to loop animation
#     date_options='YYYY-MM-DD HH:mm:ss',  # Display format for timestamps
#     time_slider_drag_update=True # Update on dragging time slider
# )

# # Initialize the Folium map centered on the dataset's centroid
# m = folium.Map(
#     location=[gdf_modis_wildfire.geometry.y.mean(), gdf_modis_wildfire.geometry.x.mean()],
#     zoom_start=6, 
#     tiles='CartoDB positron'  # Light basemap
# )

# # Add the timestamped GeoJSON to the map
# timestamped_geojson.add_to(m)

# # Save the map as an HTML file and display it
# m.save('interactive_map.html')

# # If running in Jupyter, display the map directly
# m

#### Optionally read in the HIFLD data for power grid transmission lines
uses HIFLD data: https://hifld-geoplatform.hub.arcgis.com/datasets/geoplatform::transmission-lines

In [ ]:
file_path_powergrid = "/Users/ryanmc/Documents/Conferences/Jack_Eddy_Symposium_2022/dev/physical_grid_data/U.S._Electric_Power_Transmission_Lines.geojson"
gdf_powergrid = gpd.read_file(file_path_powergrid)



In [ ]:
print('size of the power grid data is: {:.2f} MB'.format(sys.getsizeof(gdf_powergrid) / 10**6))

#### Power Grid Disturbance Data

Candidates
- [new dataset from EAGLE-I](https://www.nature.com/articles/s41597-024-03095-5) (Data are available in the Figshare repository at https://doi.org/10.6084/m9.figshare.24237376 or via archive request from the eagle-I website)
- [DOE electric disturbance events OE-417](https://www.oe.netl.doe.gov/OE417_annual_summary.aspx)


New Candidate: [NASA Black Marble](https://blackmarble.gsfc.nasa.gov/) (will require more thinking about how we might use these data)


In [ ]:
# Read in 2023 data
outage_directory = '/Users/ryanmc/Documents/Conferences/Jack_Eddy_Symposium_2022/dev/outage_data/'
oe417 = pd.read_excel(os.path.join(outage_directory,'DOE-OE-417/2023_Annual_Summary.xlsx'),header=1)
oe417 = oe417.dropna(subset=['Time Event Began'])
oe417 = oe417.iloc[:-2]
for o in range(len(oe417['Number of Customers Affected'])):
    if oe417['Number of Customers Affected'].iloc[o]=='Unknown':
        oe417['Number of Customers Affected'].iloc[o] = np.nan
    elif type(oe417['Number of Customers Affected'].iloc[o]) == str: 
        oe417['Number of Customers Affected'].iloc[o] = int(oe417['Number of Customers Affected'].iloc[o])
    
oe417 = oe417[oe417['Number of Customers Affected']>1000]

eaglei = pd.read_csv(os.path.join(outage_directory,'EAGLE-I/eaglei_outages_2023.csv'))
# limit the data to >1000 customers out
eaglei = eaglei[eaglei['customers_out']>1000]


In [ ]:
print('size of the oe417 data is: {:.2f} MB'.format(sys.getsizeof(oe417) / 10**6))
print('size of the eaglei data is: {:.2f} MB'.format(sys.getsizeof(eaglei) / 10**6))

In [ ]:
oe417

In [ ]:

from dateutil.parser import parse

def get_datetime_oe417(date,time):
    # try: 
    #     dt = datetime.combine(str_date,dt_time)
    # except: 
    #     try:
    #         date_obj = datetime.strptime(str_date, '%m/%d/%Y')
    #         # dt = datetime.combine(date_obj, dt_time)
    #         dt = parse(date_obj+' '+dt_time)
    #     except:
    #         dt = np.nan

    if type(date) == str:
        date_obj = datetime.strptime(date, '%m/%d/%Y')
        dt = datetime.combine(date_obj,time)
    else:
        dt = datetime.combine(date,time)
    
    return dt

# Regular expression to capture 'State: County' pairs
pattern = r'(\w+):\s*([\w\s-]+)'

# Function to extract states and counties from each row
# def extract_state_county_pairs(row):
#     pairs = []
#     # Split by semicolon to handle multiple state-county pairs
#     entries = row.split(';')
#     for entry in entries:
#         # Use regex to extract state and county
#         match = re.match(r'(\w+):\s*(.*)', entry.strip())
#         if match:
#             state = match.group(1)
#             county = match.group(2)
#             pairs.append((state, county))
#     return pairs

def extract_state_county_pairs(location_str):
    pairs = []

    # Split by ';' to handle multiple state-county blocks
    state_county_blocks = location_str.split(';')

    for block in state_county_blocks:
        block = block.strip()

        # Case: Handle multiple consecutive states without counties (e.g., "Alabama: Maine:")
        if re.fullmatch(r"(.+?:)+", block):
            states = [state.strip() for state in block.split(':') if state.strip()]
            for state in states:
                pairs.append((state, None))  # Store each state with None for county
            continue  # Skip further processing for this block

        # Use regex to match "State: County, County, ..." or "State:"
        match = re.match(r"(.+?):\s*(.*)", block)

        if match:
            state = match.group(1).strip()
            counties = match.group(2).strip()

            if counties:  # If counties exist, split them by ','
                counties_list = [county.strip() for county in counties.split(',')]
                for county in counties_list:
                    pairs.append((state, county))
            else:  # If no counties are provided
                pairs.append((state, None))
        else:
            # If the block contains only a state name
            pairs.append((block, None))

    return pairs


def get_oe417_geometries(row_area_affected,gdf_states,gdf_counties):
    '''
        inputs: 
            - row of oe417 data for Area Affected (oe417['Area Affected'].iloc[x])
            - data for states geometries
            - data for county geometries
        dependencies:
            - extract_state_county_pairs function
    '''
    
    row_states_counties = extract_state_county_pairs(row_area_affected)
    all_geometries = []
    for l in row_states_counties: 
        if not l[0]:
            # skip the empty rows in the extracted data
            continue
        else:
            if not l[1]:
                # if there is no county data, then just use whole state
                geom_data_tmp = gpd.GeoSeries(gdf_states['geometry'][gdf_states['NAME'] == l[0]].values)
                if geom_data_tmp.empty:
                    continue
                if geom_data_tmp.apply(lambda geom: isinstance(geom, MultiPolygon)).all():
#                     print('----> encountering multipolygon for {}, skipping'.format(l[0]))
                    all_geometries.extend([polygon for polygon in geom_data_tmp.geometry[0].geoms])
                else:
                    all_geometries.extend( [Polygon(geom_data_tmp.geometry[0])] )
            else:
                # if there is county data, get the county geometries and combine them
                fips_tmp = get_fips(l[0])
                geom_data_tmp = gpd.GeoSeries(gdf_counties['geometry'][ (gdf_counties['STATEFP'] == fips_tmp) & (gdf_counties['NAMELSAD'] == l[1]) ].values)
                if geom_data_tmp.empty:
                    continue
                if geom_data_tmp.apply(lambda geom: isinstance(geom, MultiPolygon)).all():
#                     print('----> encountering multipolygon for {}, skipping'.format(l[0]))
                    all_geometries.extend([polygon for polygon in geom_data_tmp.geometry[0].geoms])
                else:
                    all_geometries.extend( [Polygon(geom_data_tmp.geometry[0])] )

    combined_geometry = gpd.GeoSeries(all_geometries).unary_union

    return combined_geometry, row_states_counties

def get_eaglei_geometries(row_county,row_state,gdf_states,gdf_counties):
    '''
        inputs: 
            - row of eaglei data for county (eagelei['county'].iloc[x])
            - row of eaglei data for state (eagelei['state'].iloc[x])
            - data for states geometries
            - data for county geometries
        dependencies:
            - extract_state_county_pairs function
    '''
    
    all_geometries = []
    fips_tmp = get_fips(row_state)
    geom_data_tmp = gpd.GeoSeries(gdf_counties['geometry'][ (gdf_counties['STATEFP'] == fips_tmp) & (gdf_counties['NAMELSAD'] == row_county + ' County') ].values)
    if geom_data_tmp.empty:
        return np.nan
    if geom_data_tmp.apply(lambda geom: isinstance(geom, MultiPolygon)).all():
        all_geometries.extend([polygon for polygon in geom_data_tmp.geometry[0].geoms])
    else:
        all_geometries.extend( [Polygon(geom_data_tmp.geometry[0])] )
    

    combined_geometry = gpd.GeoSeries(all_geometries).unary_union

    return combined_geometry

# Dictionary mapping state abbreviations to FIPS codes
# List of tuples containing (state_abbreviation, fips_code, full_state_name)
state_fips = [
    ('AL', '01', 'Alabama'),
    ('AK', '02', 'Alaska'),
    ('AZ', '04', 'Arizona'),
    ('AR', '05', 'Arkansas'),
    ('CA', '06', 'California'),
    ('CO', '08', 'Colorado'),
    ('CT', '09', 'Connecticut'),
    ('DE', '10', 'Delaware'),
    ('FL', '12', 'Florida'),
    ('GA', '13', 'Georgia'),
    ('HI', '15', 'Hawaii'),
    ('ID', '16', 'Idaho'),
    ('IL', '17', 'Illinois'),
    ('IN', '18', 'Indiana'),
    ('IA', '19', 'Iowa'),
    ('KS', '20', 'Kansas'),
    ('KY', '21', 'Kentucky'),
    ('LA', '22', 'Louisiana'),
    ('ME', '23', 'Maine'),
    ('MD', '24', 'Maryland'),
    ('MA', '25', 'Massachusetts'),
    ('MI', '26', 'Michigan'),
    ('MN', '27', 'Minnesota'),
    ('MS', '28', 'Mississippi'),
    ('MO', '29', 'Missouri'),
    ('MT', '30', 'Montana'),
    ('NE', '31', 'Nebraska'),
    ('NV', '32', 'Nevada'),
    ('NH', '33', 'New Hampshire'),
    ('NJ', '34', 'New Jersey'),
    ('NM', '35', 'New Mexico'),
    ('NY', '36', 'New York'),
    ('NC', '37', 'North Carolina'),
    ('ND', '38', 'North Dakota'),
    ('OH', '39', 'Ohio'),
    ('OK', '40', 'Oklahoma'),
    ('OR', '41', 'Oregon'),
    ('PA', '42', 'Pennsylvania'),
    ('RI', '44', 'Rhode Island'),
    ('SC', '45', 'South Carolina'),
    ('SD', '46', 'South Dakota'),
    ('TN', '47', 'Tennessee'),
    ('TX', '48', 'Texas'),
    ('UT', '49', 'Utah'),
    ('VT', '50', 'Vermont'),
    ('VA', '51', 'Virginia'),
    ('WA', '53', 'Washington'),
    ('WV', '54', 'West Virginia'),
    ('WI', '55', 'Wisconsin'),
    ('WY', '56', 'Wyoming'),
]

def get_fips(state_name):
    for abbr, fips, name in state_fips:
        if name.upper() == state_name.upper():
            return fips
    return None, None  # Return None if not found

##### Country/State location data
Geting the data from [US Census Bureau Tiger platform](https://www.census.gov/geographies/mapping-files/time-series/geo/tiger-line-file.html)

In [ ]:
gdf_states = gpd.read_file('/Users/ryanmc/Documents/Conferences/Jack_Eddy_Symposium_2022/dev/location_data/Census_Bureau_Data/tl_2023_us_state/tl_2023_us_state.shp')
gdf_counties = gpd.read_file('/Users/ryanmc/Documents/Conferences/Jack_Eddy_Symposium_2022/dev/location_data/Census_Bureau_Data/tl_2023_us_county/tl_2023_us_county.shp')

In [ ]:
print('size of the states data is: {:.2f} MB'.format(sys.getsizeof(gdf_states) / 10**6))
print('size of the counties data is: {:.2f} MB'.format(sys.getsizeof(gdf_counties) / 10**6))


In [ ]:
# Merge into one outage data file as a gpd
outage_columns = ['outage_start_time', 'outage_stop_time', 'customers_affected','geometry','source']
outage_data_gdf = gpd.GeoDataFrame(pd.DataFrame(columns=outage_columns), geometry='geometry')

outage_data_gdf.set_crs("EPSG:4326", inplace=True)  # WGS84 - lat/lon

In [ ]:
# growing the outage data with oe417 data

for o in range(len(oe417)):
    
    # get start and end times
    str_date_start = oe417['Date Event Began'].iloc[o]
    dt_time_start = oe417['Time Event Began'].iloc[o]
    str_date_end = oe417['Date of Restoration'].iloc[o]
    dt_time_end = oe417['Time of Restoration'].iloc[o]
    try:
        dt_start = get_datetime_oe417(str_date_start,dt_time_start)
    except:
        dt_start = np.nan
    try:
        dt_end = get_datetime_oe417(str_date_end,dt_time_end)
    except:
        dt_end = np.nan    

    # get the geometries of the areas affected
    combined_geometry, states_counties = get_oe417_geometries(oe417['Area Affected'].iloc[o],gdf_states,gdf_counties)
    
    new_row = {
        'source': 'oe417', 
        'area affected': states_counties, 
        'outage_start_time': dt_start,
        'outage_stop_time': dt_end,
        'customers_affected': oe417['Number of Customers Affected'].iloc[o],
        'geometry': combined_geometry
    }
    
    outage_data_gdf = pd.concat([outage_data_gdf, pd.DataFrame([new_row])], ignore_index=True)

In [ ]:
outage_data_gdf#['outage_start_time'].isna().sum()

In [ ]:
# growing the outage data with eaglei data

for e in range(len(eaglei)):
    
    # get start times
    try:
        dt_start = pd.to_datetime(eaglei['run_start_time'].iloc[e], format='%m/%d/%y %H:%M')
    except:
        dt_start = np.nan
    
    combined_geometry = get_eaglei_geometries(eaglei['county'].iloc[e],eaglei['state'].iloc[e],gdf_states,gdf_counties)
    
    outage_data_gdf = outage_data_gdf.append({
                                                'source': 'eaglei', 
                                                'area affected': [(eaglei['state'].iloc[e],eaglei['county'].iloc[e]+' County')], 
                                                'outage_start_time': dt_start,
                                                'outage_stop_time': np.nan,
                                                'customers_affected': eaglei['customers_out'].iloc[e],
                                                'geometry': combined_geometry
                                                    }, ignore_index=True)

#### Candidate search


#### first group the modis wildfire data by datetime and nearby lat-long points


In [ ]:
import geopandas as gpd
from shapely.geometry import Point, MultiPoint

def aggregate_points_within_radius(gdf, radius=0.01):
    """
    Aggregates points by datetime and spatial proximity into polygons.

    Parameters:
    - df: GeoDataFrame with columns ["datetime", "geometry"]
    - radius: Proximity radius in degrees (~0.01 degrees ≈ 1.1 km) for grouping
    
    Returns:
    - A new GeoDataFrame with datetime and polygon geometry
    """
    # Group by datetime
    grouped = gdf.groupby('datetime')

    result_rows = []

    for datetime_value, group in grouped:
        points = group['geometry'].tolist()
        clustered_points = []

        # Track visited points
        visited = set()

        for i, point in enumerate(points):
            if i in visited:
                continue

            # Start a cluster
            cluster = [point]
            visited.add(i)

            for j, other_point in enumerate(points):
                if j not in visited and point.distance(other_point) <= radius:
                    cluster.append(other_point)
                    visited.add(j)

            # Create a polygon from the clustered points
            polygon = MultiPoint(cluster).convex_hull
            result_rows.append({
                'datetime': datetime_value,
                'geometry': polygon
            })

    # Create a new GeoDataFrame from the result
    result_gdf = gpd.GeoDataFrame(result_rows, columns=['datetime', 'geometry'], crs=gdf.crs)
    return result_gdf


# Example usage with your GeoDataFrame
gdf_modis_wildfire_result = aggregate_points_within_radius(gdf_modis_wildfire, radius=0.01)
print(gdf_modis_wildfire_result)



##### Optionally plot grouped wildfire instances in MODIS 


In [ ]:
radius = 0.1  # Example radius in degrees (adjust based on your CRS)

candidates = []
# candidate_columns = ['outage_start_time', 'outage_stop_time', 'customers_affected','geometry','source']
# candidate_gdf = gpd.GeoDataFrame(pd.DataFrame(columns=outage_columns), geometry='geometry')


# for w in range(len(gdf_modis_wildfire)):
for index_w, row_w in gdf_modis_wildfire_result.iterrows():
    
    # Identify overlap in time with the power grid disturbance
    wildfire_datetime = row_w.datetime #gdf_modis_wildfire['datetime'].iloc[w]
    filtered_outage_data_gdf = outage_data_gdf[
                                            (outage_data_gdf['outage_start_time'] >= wildfire_datetime - pd.Timedelta(hours=1)) &
                                            (outage_data_gdf['outage_start_time'] <= wildfire_datetime + pd.Timedelta(days=1))
                                            ]
    
    # Identify overlap in space with the power grid disturbance
    geom = row_w.geometry
    
    # Check if the geometry is a Point
    if geom.geom_type == 'Point':
        #   Create a buffer (circular area) around the point
        wildfire_point = gdf_modis_wildfire['geometry'].iloc[row_w.name]
        buffered_wildfire_area = geom.buffer(radius)
        disp_geom = geom.coords[0]

    # Check if the geometry is a Polygon
    elif geom.geom_type == 'Polygon':
        buffered_wildfire_area = geom
        disp_geom = geom.exterior.coords[0]

    
    for f in range(len(filtered_outage_data_gdf)):
        if buffered_wildfire_area.intersects(filtered_outage_data_gdf['geometry'].iloc[f]):
            
            # Check space weather data for space weather conditions (if at any time in the surrounding 24 hours AE exceeded 200 nT)
            sm_data = get_sm_one_minute((wildfire_datetime - pd.Timedelta(hours=24)).strftime('%Y-%m-%d %H:%M:%S'),
                                        (wildfire_datetime + pd.Timedelta(hours=12)).strftime('%Y-%m-%d %H:%M:%S'))
            
            if sm_data['SME'][sm_data['SME']>200.].any():


                # save as a candidate
                print('candidate at {}! \n wildfire: \n \t {} \n outage: \n \t {} for {} \n sp wx: \n \t {} at {}'.format(wildfire_datetime,
                                                                                                                             disp_geom,
                                                                                                                             filtered_outage_data_gdf['outage_start_time'].iloc[f],
                                                                                                                             filtered_outage_data_gdf['area affected'].iloc[f],
                                                                                                                             sm_data['SME'][sm_data['SME']==sm_data['SME'].max()].values,
                                                                                                                             sm_data.index[sm_data['SME']==sm_data['SME'].max()].values,))
                candidates.append({
                                    'wildfire datetime': wildfire_datetime,
                                    'wildfire geometry': geom,
                                    'outage_start_time': filtered_outage_data_gdf['outage_start_time'].iloc[f],
                                    'outage_stop_time': filtered_outage_data_gdf['outage_stop_time'].iloc[f],
                                    'customers affected': filtered_outage_data_gdf['customers_affected'].iloc[f],
                                    'outage geometry': filtered_outage_data_gdf['geometry'].iloc[f],
                                    'area affected': filtered_outage_data_gdf['area affected'].iloc[f],
                                    'max SME': sm_data['SME'][sm_data['SME']==sm_data['SME'].max()].values,

                                    })
                break
    continue

#### Candidate search

In [ ]:

radius = 0.1  # Example radius in degrees (adjust based on your CRS)

candidates = []
# candidate_columns = ['outage_start_time', 'outage_stop_time', 'customers_affected','geometry','source']
# candidate_gdf = gpd.GeoDataFrame(pd.DataFrame(columns=outage_columns), geometry='geometry')


# for w in range(len(gdf_modis_wildfire)):
for index_w, row_w in gdf_modis_wildfire_result.iterrows():
    
    # Identify overlap in time with the power grid disturbance
    wildfire_datetime = row_w.datetime #gdf_modis_wildfire['datetime'].iloc[w]
    filtered_outage_data_gdf = outage_data_gdf[
                                            (outage_data_gdf['outage_start_time'] >= wildfire_datetime - pd.Timedelta(hours=1)) &
                                            (outage_data_gdf['outage_start_time'] <= wildfire_datetime + pd.Timedelta(days=1))
                                            ]
    
    # Identify overlap in space with the power grid disturbance
    geom = row_w.geometry
    
    # Check if the geometry is a Point
    if geom.geom_type == 'Point':
        #   Create a buffer (circular area) around the point
        wildfire_point = gdf_modis_wildfire['geometry'].iloc[row_w.name]
        buffered_wildfire_area = geom.buffer(radius)
        disp_geom = geom.coords[0]

    # Check if the geometry is a Polygon
    elif geom.geom_type == 'Polygon':
        buffered_wildfire_area = geom
        disp_geom = geom.exterior.coords[0]

    
    for f in range(len(filtered_outage_data_gdf)):
        if buffered_wildfire_area.intersects(filtered_outage_data_gdf['geometry'].iloc[f]):
            
            # Check space weather data for space weather conditions (if at any time in the surrounding 24 hours AE exceeded 200 nT)
            sm_data = get_sm_one_minute((wildfire_datetime - pd.Timedelta(hours=24)).strftime('%Y-%-m-%-d %H:%M:%S'),
                                        (wildfire_datetime + pd.Timedelta(hours=12)).strftime('%Y-%-m-%-d %H:%M:%S'))
            
            if sm_data['SME'][sm_data['SME']>200.].any():


                # save as a candidate
                print('candidate at {}! \n wildfire: \n \t {} \n outage: \n \t {} for {} \n sp wx: \n \t {} at {}'.format(wildfire_datetime,
                                                                                                                             disp_geom,
                                                                                                                             filtered_outage_data_gdf['outage_start_time'].iloc[f],
                                                                                                                             filtered_outage_data_gdf['area affected'].iloc[f],
                                                                                                                             sm_data['SME'][sm_data['SME']==sm_data['SME'].max()].values,
                                                                                                                             sm_data.index[sm_data['SME']==sm_data['SME'].max()].values,))
                candidates.append({
                                    'wildfire datetime': wildfire_datetime,
                                    'wildfire geometry': geom,
                                    'outage_start_time': filtered_outage_data_gdf['outage_start_time'].iloc[f],
                                    'outage_stop_time': filtered_outage_data_gdf['outage_stop_time'].iloc[f],
                                    'customers affected': filtered_outage_data_gdf['customers_affected'].iloc[f],
                                    'outage geometry': filtered_outage_data_gdf['geometry'].iloc[f],
                                    'area affected': filtered_outage_data_gdf['area affected'].iloc[f],
                                    'max SME': sm_data['SME'][sm_data['SME']==sm_data['SME'].max()].values,

                                    })
                break
    continue
                
            


In [ ]:
candidates

In [ ]:
gdf_candidates

In [ ]:
gdf_candidates = gpd.GeoDataFrame(candidates)
gdf_candidates = gdf_candidates.set_geometry("outage geometry")


# Save with pickle 
gdf_candidates.to_pickle("/Users/ryanmc/Documents/Conferences/Jack_Eddy_Symposium_2022/dev/candidate_multihazards_data/wildfire_space-weather_2023_candidates.pkl")
# loaded_gdf = gpd.read_pickle("hazard_events.pkl")

# # Save to GeoJSON (need to ensure all geometry types are the same before this works)
# gdf_candidates.to_file("/Users/ryanmc/Documents/Conferences/Jack_Eddy_Symposium_2022/dev/candidate_multihazards_data/wildfire_space-weather_2023_candidates.geojson", driver="GeoJSON")

#### Load EM-DAT data

In [ ]:
directory_emdat = '/Users/ryanmc/Documents/Conferences/Jack_Eddy_Symposium_2022/dev/emdat_data/'
file_emdat = 'public_emdat_2023_US.xlsx'
df_emdat = pd.read_excel(os.path.join(directory_emdat,file_emdat))

In [ ]:
df_emdat.columns.tolist()

In [ ]:
df_emdat[['Start Year', 'Start Month', 'Start Day']]
df_emdat[['End Year', 'End Month', 'End Day']]



In [ ]:
df_emdat['start_datetime'] = df_emdat.apply(
    lambda row: pd.Timestamp(int(row['Start Year']), int(row['Start Month']), int(row['Start Day'])) 
    if pd.notna(row['Start Year']) and pd.notna(row['Start Month']) and pd.notna(row['Start Day'])
    else pd.NaT,
    axis=1
)

df_emdat['end_datetime'] = df_emdat.apply(
    lambda row: pd.Timestamp(int(row['End Year']), int(row['End Month']), int(row['End Day'])) 
    if pd.notna(row['End Year']) and pd.notna(row['End Month']) and pd.notna(row['End Day'])
    else pd.NaT,
    axis=1
)



In [ ]:
gdf_candidates.columns.tolist() #

#### Compare potential events across candidates and database(s)

In [ ]:
for idx,row_emdat in df_emdat.iterrows():
    tmp_diff = (gdf_candidates['outage_start_time'] - row_emdat['start_datetime']).abs()
    result = gdf_candidates[tmp_diff <= pd.Timedelta(days=1)]

    print('overlap of time around {}, \n \t location candidate = {} \n \t  location emdat = {}'.format(row_emdat['start_datetime'],result['area affected'],row_emdat['Location']))
    print('\n\n')


    

Check on


overlap of time around 2023-04-01 00:00:00, 
 	 location candidate = 10    [(Arkansas, None)]
11    [(Illinois, None)]
Name: area affected, dtype: object 
 	  location emdat = Texas, Louisiana, Oklahoma, Kansas, Illinois, Missouri, Nebraska, Washington, Oregon, Montana





In [ ]:
gdf_candidates_multihazard = gdf_candidates[(gdf_candidates['outage_start_time'] >= datetime(2023,3,31)) & (gdf_candidates['outage_start_time'] <= datetime(2023,4,2))]


In [ ]:
# import geopandas as gpd
# import folium
# import time
# from shapely.geometry import Point, Polygon
# from IPython.display import display, clear_output


# Create a Folium map centered over CONUS
conus_map = folium.Map(location=[37.5, -95], zoom_start=5)

for index, row in gdf_candidates_multihazard.iterrows():
    geom = row['wildfire geometry']

    # Check if the geometry is a Point
    if geom.geom_type == 'Point':
        marker_layer = folium.Marker(
            location=[geom.y, geom.x],
            popup="Point Geometry"
        ).add_to(conus_map)
    
    # Check if the geometry is a Polygon
    elif geom.geom_type == 'Polygon':
        poly_layer = folium.Polygon(
            locations=[(y, x) for x, y in geom.exterior.coords],
            color="blue",
            fill=True,
            fill_opacity=0.4,
            popup="Polygon Geometry"
        ).add_to(conus_map)

display(conus_map)


In [ ]:
for index, row in gdf_candidates_multihazard.iterrows():
    print(row)

In [ ]:
row['outage geometry']

TODO
- Compile the concrete information that we know about this event
- Determine what continuous data we have (imagery over time, space weather data (GIC and SuperMAG)
- Can these 'layers' (need the weather and space weather spatiotemporal maps) yield network data and how might I explore its information over time?
- get additional data - Geo-electric field
- ...